#### Master 1ère année
#### Initiation à la Cryptographie

# Chiffrements antiques 

In [286]:
import numpy as np
import string

### [César]

On considère le chiffrement par décalage de d positions des lettres de l'alphabet. Si
d = 3, le chiffrement est appelé Chiffre de César et ainsi la lettre A devient D, la lettre B devient E,
etc

In [14]:
def chiffrerCesar(clair,clé):
    chiffre=""
    for lettre in clair:
        val=ord(lettre)+clé
        chiffre+=chr(val)
    return chiffre

def decffirerCesar(chiffré,clé):
    clair=""
    for lettre in chiffré:
        val=ord(lettre)-clé
        clair+=chr(val)
    return clair


### 1 - Chiffrer le message suivant ****CESAR VAINCRA ***

In [13]:
msg="cesar vaincra"
d=3

"Le chiffré est : ",chiffrerCesar(msg,d)

('Le chiffré est : ', 'fhvdu#ydlqfud')

### 2 - Déchiffrer le message suivant  ***DOHDMDFWDHVW***

In [16]:
chiffré="DOHDMDFWDHVW"
" Le clair est : ",decffirerCesar(chiffré,d)

(' Le clair est : ', 'ALEAJACTAEST')

### On peut généraliser par une substitution quelconque : on fait correspondre l'alphabet ordonné avec un alphabet permuté. Combien y-a-t-il de possibilités?

R : 26!-1 possibilités , car on prend pas la permuntation à une lettre

### [Carré de Polybe] 

Le chiffrement est effectué à l'aide d'une matrice indexée par les entiers
de 1 à 5 dans laquelle on met les 26 lettres de l'alphabet (W supprimé ou I et J sont dans la même case).
Par exemple, on considère la matrice suivante :


|   | 0 | 1 | 2 | 3 | 4 |
|:-:|:-:|:-:|:-:|:-:|:-:|
| 0 | A | B | C | D | E |
| 1 | F | G | H | I | J |
| 2 | K | L | M | N | O |
| 3 | P | Q | R | S | T |
| 4 | U | V | X | Y | Z |

In [127]:
def ajouterCléAuCarre(clé):
    carre=np.zeros((5,5))
    lig=0
    col=0
    for lettre in clé:
        if ord(lettre.upper()) not in carre:
            if col==5: # au bout d'une ligne 
                lig+=1
                col=0
                
            if lig==5:
                break
                
            carre[lig][col]=ord(lettre.upper())
            col+=1
    return carre,lig,col
    
def terminerLeCarre(carre,lig,col):
    col=col
    lig=lig
    alph=["a", "b", "c", "d","e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "x", "y" , "z"]
    for lettre in alph:
        if ord(lettre.upper()) not in carre:
            if col==5: # au bout d'une ligne 
                lig+=1
                col=0
                
            if lig==5:
                break
                
            carre[lig][col]=ord(lettre.upper())
            col+=1
    return carre


def findLettre(lettre,carre):
    res=""
    for lig in range(carre.shape[0]):
        find = False 
        for col in range(carre.shape[1]):
            if lettre==carre[lig][col]:
                res+=str(lig)+str(col)
                find=True
                break
        if find:
            break
    return res+"-"
            
            
            
def chifferPolybe(clair,carre):
    res=""
    for lettre in clair:
        res+=findLettre(ord(lettre.upper()),carre)
    return res


def dechiffrerPolybe(chiffre,carre):
    res=""
    for item in chiffre.split("-"):
        if item:
            lig=int(item[0])
            col=int(item[1])
            val=carre[lig][col]
            #print(int(val),"<<")
            res+=chr(int(carre[lig][col]))
    return res
    
        

In [128]:
carre,lig,col=ajouterCléAuCarre("abcde")
terminerLeCarre(carre,lig,col)
clair="CarreDEPolybe"
chiffré=chifferPolybe(clair,carre)
" Le chiffre est : ",chifferPolybe(clair,carre)

(' Le chiffre est : ', '02-00-32-32-04-03-04-30-24-21-43-01-04-')

In [131]:
" Le déchiffé est : ",dechiffrerPolybe(chiffré,carre)

(' Le déchiffé est : ', 'CARREDEPOLYBE')

### [Permutation] 

On considère le chiffrement par une écriture en dent de scie : on prend une
lettre sur deux du mot puis le reste

In [276]:
def chiffrementDentdeScie(clair,d):
    clair=list(clair)
    res=""
    for i in range(0,len(clair),d):
        res+=clair[i]
        
    for i in range(1,len(clair),2):
        res+=clair[i]
    return res

def dechiffrerDentdeScie(chiffré):
    res=[""]*len(chiffré)
    tailleTotal=len(chiffré)
    moitié2=int(tailleTotal/2)
    moitié1=tailleTotal-moitié2
    mot1 = chiffré[:moitié1]
    mot2 = chiffré[moitié2+1:]
    for i in range(0,moitié1):
        res[2*i]=mot1[i]
    for i in range(1,moitié2+1):
           res[2*i-1]=mot2[i-1]
        
    
    return res

### 1 - Chiffrer le message suivant ***MASTERLIMOGES***.

In [277]:
clair="masterlimoges"
chiffré=chiffrementDentdeScie(clair,2)
" Le chiffré est : ",chiffré

(' Le chiffré est : ', 'mselmgsatrioe')

In [278]:
" Le dechiffré est : ",dechiffrerDentdeScie(chiffré)

(' Le dechiffré est : ',
 ['m', 'a', 's', 't', 'e', 'r', 'l', 'i', 'm', 'o', 'g', 'e', 's'])

## [Vigenère, ≈ 1550]

Chaque lettre du message est déplacé dans l'alphabet selon un vecteur
décalage de longueur k, qui est utilisé comme clef (A=1). Ce vecteur est répété si nécessaire.

In [337]:
def decoupage(chaine,cle):
    chaine = list(chaine)
    decoupage=[]
    if len(cle) >1:
        while len(chaine)>0:
            lTemp=[]
            for i in range(len(cle)):
                if i <len(chaine):
                    lTemp.append(chaine[i])
            decoupage.append(lTemp)        
            for i in range(len(cle)):
                if len(chaine)>0:
                    del chaine[0]
    return decoupage

def chiffrer(chaine,cle):
    cypherText=""
    alph=list(string.ascii_uppercase)
    decoup = decoupage(chaine,cle)
    for bout in decoup:
        morceau=""
        for i in range(len(bout)):
            indexMot=alph.index(bout[i].upper())
            indexCle=alph.index(cle[i].upper())
            pos= indexMot+indexCle
            pos=pos%26
            morceau+=alph[pos]
        cypherText+=morceau
    return cypherText
    
def dechiffrer(chaine,cle):
    plainText=""
    alph=list(string.ascii_uppercase)
    decoup= decoupage(chaine,cle)
    for bout in decoup:
        morceau=""
        for i in range(len(bout)):
            indexMot=alph.index(bout[i].upper())
            indexCle=alph.index(cle[i].upper())
            pos= indexMot-indexCle
            pos=pos%26
            morceau+=alph[pos]
        plainText+=morceau
    return plainText

In [342]:
clé="master"
clair="vigenere"
chiffré = chiffrer(clair,clé)
" Le chiffré est : ",chiffré

(' Le chiffré est : ', 'HIYXRVDE')

In [343]:
dechiffré = dechiffrer(chiffré,clé)
" Le déchiffré est : ", dechiffré

(' Le déchiffré est : ', 'VIGENERE')

### [Playfair, 1854]

Le chiffre de Playfair utilise un tableau de 5x5 lettres, contenant un mot
clé ou une phrase. La mémorisation du mot clé et de 4 règles faciles à suivre suffisait pour utiliser ce
chiffrement.

Pour construire le tableau, on devait d'abord le remplir avec les lettres du mot clé (en ignorant les
doublons), puis le compléter avec les autres lettres de l'alphabet dans l'ordre (soit en omettant la lettre
W peu courante, soit en occupant une même case pour les lettres I et J suivant les versions). Le mot clé
peut être écrit en ligne, en colonne ou même en spirale (on choisira la version ligne).
Pour chiffrer un message, il faut prendre les lettres 2 par 2 et appliquer les règles suivantes en fonction
de la position des lettres dans la table :

 -  si les 2 lettres sont identiques (ou s'il n'en reste qu'une) mettre un 'X' après la première lettre. Chiffrer la nouvelle paire ainsi constituée et continuer avec la suivante.

 - si les lettres se trouvent sur la même ligne de la table, il faut les remplacer par celles se trouvant immédiatement à leur droite (en bouclant sur la gauche si le bord est atteint),

- si les lettres apparaissent sur la même colonne, les remplacer par celles qui sont juste en dessous(en bouclant par le haut si le bas de la table est atteint),

- sinon, remplacer les lettres par celles se trouvant sur la même ligne, mais dans le coin opposé du rectangle défini par la paire originale.

In [345]:
carre,lig,col=ajouterCléAuCarre("PLAYFAIR")
clé = terminerLeCarre(carre,lig,col)
clé

array([[80., 76., 65., 89., 70.],
       [73., 82., 66., 67., 68.],
       [69., 71., 72., 74., 75.],
       [77., 78., 79., 81., 83.],
       [84., 85., 86., 88., 90.]])

In [346]:
def chiffrerPlayFair(clair,carre):
    print(" à terminer hahahaha")
    

'P'